In [1]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [3]:
%matplotlib

Using matplotlib backend: <object object at 0x0000025E36ECCA50>


In [9]:
#model response
m = 0.002
R = 0.02
g = 9.8
L = 0.20
d = 0.12
J = (2/3)*m*R**2
s = ct.tf('s')
G = ((m*g*d)/(L*((J/R**2)+m)))*(1/s**2)

Kp = 6
Ki = 0.0005
Kd = 7

C = Kp+Ki/s+Kd*s
print(G)
plant = ct.feedback(G*C)
print(plant)
T,U = ct.step_response(-15*plant,T = 2)
fig8,ax8 = plt.subplots()

L = len(U)
zeros = []
for i in range(0,L-1):
    if abs((U[-1]-U[i-1])/U[-1])>0.05 and abs((U[-1]-U[i+1])/U[-1])<0.05:
        zeros.append(T[i])
for i in range(0,L):
    if (U[-1]-U[i])/U[-1]<0.05 and T[i]>=zeros[-1]:
        I = i
        sett=T[i]
        Usett = U[i]
        break

dict = ct.step_info(15*plant)
print(dict)
over = dict['Overshoot']
print('Overshoot = ',over,'%')
rise = dict['RiseTime']
print('Rise time =',rise,'s')
print("Settling time =",sett,'s')        

ax8.plot(T,U+30,label = 'response')
ax8.axhline(15,color = 'red',linestyle ='dotted',label = 'setpoint')
ax8.axhline(U[-1]*0.95+30,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax8.axhline(U[-1]*1.05+30,color = 'green',linestyle = 'dotted',label = '5% upper')
ax8.grid()
ax8.plot(sett,Usett+30,'*',label = "settling point",color = 'r')
#ax8.vlines(0.5,ymin=15,ymax = U[-1],color='r',label = "steady state error")
ax8.vlines(T[np.argmin(U)],ymin=np.min(U)+30,ymax=15,label = 'Overshoot', color = 'red')
ax8.legend()
ax8.set_ylabel('Displacement [cm]',fontsize = 15)
ax8.set_xlabel('Time [s]',fontsize = 15)
fig8.suptitle('Manually-Tuned Model Step Response for Ping Pong Ball',fontsize = 15)



3.528
-----
 s^2


   24.7 s^2 + 21.17 s + 0.001764
-----------------------------------
s^3 + 24.7 s^2 + 21.17 s + 0.001764

{'RiseTime': 0.08206498538521813, 'SettlingTime': nan, 'SettlingMin': 13.617184514329228, 'SettlingMax': 15.434099850482726, 'Overshoot': 2.893999003218172, 'Undershoot': 0, 'Peak': 15.434099850482726, 'PeakTime': 0.28722744884826346, 'SteadyStateValue': 15.0}
Overshoot =  2.893999003218172 %
Rise time = 0.08206498538521813 s
Settling time = 0.11014492753623188 s


Text(0.5, 0.98, 'Manually-Tuned Model Step Response for Ping Pong Ball')

In [5]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figz,axz = plt.subplots(5,1,figsize = (6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)

fig.suptitle('Individual Manually Tuned Responses for Ping Pong Ball')
figraw.suptitle('Unfiltered Manually Tuned Responses for Ping Pong Ball')
figz.suptitle('Motor Angle for Manually Tuned Ping Pong Ball Response')

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Manually Tuned Ping Pong Ball Responses')
#fig3.suptitle('Unfiltered Average of Manually Tuned Ping Pong Ball Responses')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Distance [cm]')
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1517
for i in range(1,6):
    df = pd.read_csv('PING_MAT_MAN_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs[i-1].plot(timenew,filtered,label = 'filtered',color = 'b')
    axs[i-1].plot(timenew,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1].set_xlabel('Time [s]')
    axs[i-1].set_ylabel('Distance [cm]')
    axs[i-1].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1].grid()
    
    axsraw[i-1].plot(timenew,distance)
    axsraw[i-1].set_xlabel('Time [s]')
    axsraw[i-1].set_ylabel('Distance [cm]')
    axsraw[i-1].grid()
    
    axz[i-1].plot(timenew,angle_filt,label = 'filtered',color = 'b')
    axz[i-1].plot(timenew,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1].set_xlabel('Time [s]')
    axz[i-1].set_ylabel('Servo Angle [degrees]')
    axz[i-1].grid()
    axz[i-1].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axs[0].legend(loc = 'lower right')
axz[0].legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,32,33)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1560
1520
1889
1739
1517
[0.015, 0.034600000000000006, 0.053000000000000005, 0.07160000000000001, 0.09159999999999999, 0.10980000000000001, 0.12940000000000002, 0.1476, 0.16760000000000003, 0.18579999999999997, 0.2052, 0.2242, 0.2424, 0.262, 0.28020000000000006, 0.29999999999999993, 0.31799999999999995, 0.3378, 0.35619999999999996, 0.37560000000000004, 0.39459999999999995, 0.4128, 0.4326, 0.4506, 0.4702, 0.4888, 0.5077999999999999, 0.5266, 0.5458000000000001, 0.5645999999999999, 0.5828, 0.6028, 0.6212, 0.6406000000000001, 0.6596, 0.679, 0.6978, 0.7169999999999999, 0.7356, 0.7544000000000001, 0.7736, 0.792, 0.8109999999999999, 0.8299999999999998, 0.849, 0.8674, 0.8864000000000001, 0.9052, 0.9236000000000001, 0.9428000000000001, 0.9611999999999998, 0.9805999999999999, 0.9991999999999999, 1.0181999999999998, 1.0364, 1.0554000000000001, 1.0736, 1.092, 1.1108, 1.129, 1.1476, 1.1658000000000002, 1.1844, 1.2027999999999999, 1.2209999999999999, 1.2398000000000002, 1.2580000000000002, 1.2766000

In [5]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual Manually Tuned Responses for Ping Pong Ball',fontsize = 15)
figraw.suptitle('Unfiltered Manually Tuned Responses for Ping Pong Ball',fontsize = 15)
figz.suptitle('Motor Angle for Manually Tuned Ping Pong Ball Response',fontsize = 15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Manually Tuned Ping Pong Ball Responses',fontsize = 15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Ping Pong Ball Responses')
ax2.set_xlabel('Time [s]',fontsize = 15)
ax2.set_ylabel('Distance [cm]',fontsize = 15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')
lines = ['solid','dashed','dotted','dashdot','solid']

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1517
ticksang = np.linspace(-30,30,11)
for i in range(1,6):
    df = pd.read_csv('PING_MAT_MAN_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]',fontsize = 15)
    axs.set_ylabel('Distance [cm]',fontsize = 15)
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize = 15)
    axsraw.set_ylabel('Distance [cm]',fontsize = 15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize = 15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize = 15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right',fontsize = 15)
axz.legend(loc = 'lower right',fontsize = 15)
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1560
1520
1889
1739
1517
[0.015, 0.034600000000000006, 0.053000000000000005, 0.07160000000000001, 0.09159999999999999, 0.10980000000000001, 0.12940000000000002, 0.1476, 0.16760000000000003, 0.18579999999999997, 0.2052, 0.2242, 0.2424, 0.262, 0.28020000000000006, 0.29999999999999993, 0.31799999999999995, 0.3378, 0.35619999999999996, 0.37560000000000004, 0.39459999999999995, 0.4128, 0.4326, 0.4506, 0.4702, 0.4888, 0.5077999999999999, 0.5266, 0.5458000000000001, 0.5645999999999999, 0.5828, 0.6028, 0.6212, 0.6406000000000001, 0.6596, 0.679, 0.6978, 0.7169999999999999, 0.7356, 0.7544000000000001, 0.7736, 0.792, 0.8109999999999999, 0.8299999999999998, 0.849, 0.8674, 0.8864000000000001, 0.9052, 0.9236000000000001, 0.9428000000000001, 0.9611999999999998, 0.9805999999999999, 0.9991999999999999, 1.0181999999999998, 1.0364, 1.0554000000000001, 1.0736, 1.092, 1.1108, 1.129, 1.1476, 1.1658000000000002, 1.1844, 1.2027999999999999, 1.2209999999999999, 1.2398000000000002, 1.2580000000000002, 1.2766000